<a href="https://colab.research.google.com/github/agolia14/Coupan-Redemption-Prediction/blob/Data-Exploration/Customer_Transaction_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from collections import Counter
import pickle

from sklearn.preprocessing import LabelEncoder

In [2]:
!git init
!git config --global user.email "agolia14@gmail.com"

!curl -s https://packagecloud.io/install/repositories/github/git-lfs/script.deb.sh | sudo bash
!sudo apt-get install git-lfs

!git remote add origin https://d0e5c5d2c5891214a6eb9a978deb96f359421b01@github.com/agolia14/Coupan-Redemption-Prediction.git
!git fetch
!git checkout Data-Exploration

'git' is not recognized as an internal or external command,
operable program or batch file.
'git' is not recognized as an internal or external command,
operable program or batch file.
'curl' is not recognized as an internal or external command,
operable program or batch file.
'sudo' is not recognized as an internal or external command,
operable program or batch file.
'git' is not recognized as an internal or external command,
operable program or batch file.
'git' is not recognized as an internal or external command,
operable program or batch file.
'git' is not recognized as an internal or external command,
operable program or batch file.


## Make it Code Cell Only if GIT is not ready for large files
!git lfs install

!git lfs track "*.npy"

!git add .gitattributes

!git commit -m "Updating git Attributes"

!git push origin

In [9]:
raw = pd.read_csv("data/train/customer_transaction_data.csv", parse_dates=['date'], infer_datetime_format=True)
raw.head()

,date,customer_id,item_id,quantity,selling_price,other_discount,coupon_discount
0,2012-01-02,1501,26830,1,35.26,-10.69,0.0
1,2012-01-02,1501,54253,1,53.43,-13.89,0.0
2,2012-01-02,1501,31962,1,106.50,-14.25,0.0
3,2012-01-02,1501,33647,1,67.32,0.00,0.0
4,2012-01-02,1501,48199,1,71.24,-28.14,0.0


In [ ]:
data = pd.concat([raw,
           pd.DataFrame(columns=["day","month","year"])],
         axis=1)

data.day = [date.day for date in data.date]
data.month = [date.month for date in data.date]
data.year = [date.year-2000 for date in data.date]

data.head()

In [ ]:
data.dtypes

In [ ]:
data.info()

In [ ]:
npFile = open("npy/customer_transaction_data.npy","wb")
pickle.dump(data, npFile)
npFile.close()

In [ ]:
#!git add npy/customer_transaction_data.npy
#!git commit -m "Adding Campaign Data npy"
#!git push origin